# Setup

In [1]:
import os.path as osp

In [2]:
cfg="../usage-docs/sample-configs/datasets/config-veri.yml"
osp.exists(cfg)

True

In [3]:
from ednaml.core import EdnaML

## Define Crawler Class

In [4]:
from collections import defaultdict
import os,re,glob,xml.etree.ElementTree as ET
class CustomVeRiCrawler:
	def __init__(A,data_folder='VeRi',train_folder='image_train',test_folder='image_test',query_folder='image_query',**B):A.metadata={};A.data_folder=data_folder;A.train_folder=os.path.join(A.data_folder,train_folder);A.test_folder=os.path.join(A.data_folder,test_folder);A.query_folder=os.path.join(A.data_folder,query_folder);A.tracks_file=os.path.join(A.data_folder,'test_track.txt');A.testlabel=os.path.join(A.data_folder,'test_label.xml');A.trainlabel=os.path.join(A.data_folder,'train_label.xml');A.__verify(A.data_folder);A.__verify(A.train_folder);A.__verify(A.test_folder);A.__verify(A.query_folder);A.crawl()
	def __verify(B,folder):
		A=folder
		if not os.path.exists(A):raise IOError('Folder {data_folder} does not exist'.format(data_folder=A))
		else:0
	def crawl(A):
		A.colordict=defaultdict(lambda:-1);A.typedict=defaultdict(lambda:-1)
		with open(A.trainlabel,'r')as F:E=ET.fromstring(F.read())
		for B in E[0]:D=B.get('vehicleID');G=B.get('colorID');H=B.get('typeID');A.colordict[int(D)]=int(G)-1;A.typedict[int(D)]=int(H)-1
		with open(A.testlabel,'r')as F:E=ET.fromstring(F.read())
		for B in E[0]:D=B.get('vehicleID');G=B.get('colorID');H=B.get('typeID');A.colordict[int(D)]=int(G)-1;A.typedict[int(D)]=int(H)-1
		del E;A.classes={};A.classes['color']=10;A.classes['type']=9;A.metadata['train'],A.metadata['test'],A.metadata['query'],A.metadata['track']={},{},{},{};A.metadata['train']['crawl'],A.metadata['train']['pid'],A.metadata['train']['cid'],A.metadata['train']['imgs']=A.__crawl(A.train_folder,reset_labels=True);A.metadata['test']['crawl'],A.metadata['test']['pid'],A.metadata['test']['cid'],A.metadata['test']['imgs']=A.__crawl(A.test_folder);A.metadata['query']['crawl'],A.metadata['query']['pid'],A.metadata['query']['cid'],A.metadata['query']['imgs']=A.__crawl(A.query_folder);A.metadata['track']['crawl'],A.metadata['track']['pid'],A.metadata['track']['cid'],A.metadata['track']['imgs'],A.metadata['track']['dict'],A.metadata['track']['info']=A.__crawltracks(A.test_folder);A.metadata['val'],A.metadata['full']={},{};A.metadata['val']['crawl'],A.metadata['full']['crawl']=[],[]
		for C in ['train','test','val','full']:A.metadata[C]['imgs']=len(A.metadata[C]['crawl']);A.metadata[C]['classes']={};A.metadata[C]['classes']['color']=10;A.metadata[C]['classes']['type']=9;A.metadata[C]['classes']['pid']=A.metadata['train']['pid'];A.metadata[C]['classes']['cid']=A.metadata['train']['cid']
		print('Train\tPID: {:6d}\tCID: {:6d}\tIMGS: {:8d}'.format(A.metadata['train']['pid'],A.metadata['train']['cid'],A.metadata['train']['imgs']));print('Test \tPID: {:6d}\tCID: {:6d}\tIMGS: {:8d}'.format(A.metadata['test']['pid'],A.metadata['test']['cid'],A.metadata['test']['imgs']));print('Query\tPID: {:6d}\tCID: {:6d}\tIMGS: {:8d}'.format(A.metadata['query']['pid'],A.metadata['query']['cid'],A.metadata['query']['imgs']));print('Tracks\tPID: {:6d}\tCID: {:6d}\\Tracks: {:8d}'.format(A.metadata['track']['pid'],A.metadata['track']['cid'],A.metadata['track']['imgs']))
	def __crawl(F,folder,reset_labels=False):
		I=glob.glob(os.path.join(folder,'*.jpg'));J=re.compile('([\\d]+)_[a-z]([\\d]+)');G=0;C,D={},{};E=[];K,L,M=0,0,0
		for H in I:
			A,B=map(int,J.search(H).groups())
			if A<0:continue
			if B<0:continue
			if A not in C:C[A]=G if reset_labels else A;G+=1
			if B not in D:D[B]=B-1
			E.append((H,C[A],B-1,F.colordict[A],F.typedict[A]))
		return E,len(C),len(D),len(E)
	def __crawltracks(D,folder,reset_labels=False):
		L=re.compile('([\\d]+)_[a-z]([\\d]+)');H=0;B,M={},{};E=[];Q,R,S=0,0,0;I,J={},{};F=0
		with open(D.tracks_file,'r')as N:
			for O in N:
				K=O.strip().split(' ');T=K[0];C=K[1:];C=[os.path.join(folder,A)for A in C];A,G=map(int,L.search(C[0]).groups())
				if A<0:continue
				if G<0:continue
				if A not in B:B[A]=H if reset_labels else A;H+=1
				E.append((C,B[A],G-1,D.colordict[A],D.typedict[A]))
				for P in C:I[P]=F
				J[F]={'pid':B[A],'cid':G-1};F+=1
		return E,len(B),len(M),len(E),I,J

## define arguments, type, and instance

In [5]:
crawlerclass = CustomVeRiCrawler
crawler_args={"data_folder":"../Data/VeRi"}
crawler_instance = CustomVeRiCrawler(**crawler_args)

Train	PID:    575	CID:     20	IMGS:    37746
Test 	PID:    200	CID:     19	IMGS:    11579
Query	PID:    200	CID:     19	IMGS:     1678
Tracks	PID:    200	CID:      0\Tracks:     2021


## Approach 1 -- pass crawler class and arguments

In [6]:
eml = EdnaML(cfg, "train")

In [7]:
eml.cfg.TRANSFORMATION.BATCH_SIZE=64
eml.cfg.EXECUTION.SKIPEVAL=False

In [9]:
eml.addCrawlerClass(crawlerclass, **crawler_args)

In [10]:
eml.apply()

11:26:46 ****************************************
11:26:46 
11:26:46 
11:26:46 Using the following configuration:
11:26:46 EXECUTION:
  DATAREADER:
    CRAWLER_ARGS:
      data_folder: Data/VeRi
    DATALOADER_ARGS: {}
    DATAREADER: VeRi
    DATASET_ARGS:
      annotationidx: 3
      classificationclass: color
      pathidx: 0
    GENERATOR: ClassificationGenerator
    GENERATOR_ARGS: {}
  EPOCHS: 10
  FP16: false
  MODEL_SERVING: None
  OPTIMIZER_BUILDER: ClassificationOptimizer
  SKIPEVAL: false
  TEST_FREQUENCY: 5
  TRAINER: ClassificationTrainer
LOGGING:
  STEP_VERBOSE: 100
LOSS:
- KWARGS:
  - {}
  LABEL: color
  LAMBDAS:
  - 1.0
  LOSSES:
  - SoftmaxLogitsLoss
  NAME: out1
LOSS_OPTIMIZER:
- BASE_LR: 0.001
  LR_BIAS_FACTOR: 1.0
  OPTIMIZER: Adam
  OPTIMIZER_KWARGS: {}
  OPTIMIZER_NAME: opt-1
  WEIGHT_BIAS_FACTOR: 0.0005
  WEIGHT_DECAY: 0.0005
LOSS_SCHEDULER:
- LR_KWARGS:
    step_size: 20
  LR_SCHEDULER: StepLR
  SCHEDULER_NAME: opt-1
MODEL:
  BUILDER: classification_model_builde

Train	PID:    575	CID:     20	IMGS:    37746
Test 	PID:    200	CID:     19	IMGS:    11579
Query	PID:    200	CID:     19	IMGS:     1678
Tracks	PID:    200	CID:      0\Tracks:     2021


11:26:46 Finished instantiating model with ClassificationResnet architecture
11:26:46 No saved model weights provided.
11:26:52 ==========================================================================================================================================================================
Layer (type:depth-idx)                        Input Shape               Output Shape              Param #                   Kernel Shape              Mult-Adds
ClassificationResnet                          --                        --                        --                        --                        --
├─resnet: 1-1                                 [64, 3, 100, 100]         [64, 512, 7, 7]           --                        --                        --
│    └─Conv2d: 2-1                            [64, 3, 100, 100]         [64, 64, 50, 50]          9,408                     [3, 64, 7, 7]             1,505,280,000
│    └─BatchNorm2d: 2-2                       [64, 64, 50, 50]         

In [11]:
eml.train()

11:26:57 Starting training
11:26:57 Logging to:	colabel_devel-v1-res18-type-logger.log
11:26:57 Models will be saved to local directory:	colabel_devel-v1-res18-type
11:26:57 Models will be saved with base name:	colabel_devel-v1_epoch[].pth
11:26:57 Optimizers will be saved with base name:	colabel_devel-v1_epoch[]_optimizer.pth
11:26:57 Schedulers will be saved with base name:	colabel_devel-v1_epoch[]_scheduler.pth
11:26:57 Performing initial evaluation...


KeyboardInterrupt: 

## Approach 2 -- pass crawler instance directly

In [12]:
eml = EdnaML(cfg, "train")

Log file exists at colabel_devel-v1-res18-type\colabel_devel-v1-res18-type-logger.log. Will attempt to append there.


In [13]:
eml.cfg.TRANSFORMATION.BATCH_SIZE=64
eml.cfg.EXECUTION.SKIPEVAL=False

In [14]:
eml.addCrawler(crawler_instance)

In [15]:
eml.apply()

11:27:18 ****************************************
11:27:18 
11:27:18 
11:27:18 Using the following configuration:
11:27:18 EXECUTION:
  DATAREADER:
    CRAWLER_ARGS:
      data_folder: Data/VeRi
    DATALOADER_ARGS: {}
    DATAREADER: VeRi
    DATASET_ARGS:
      annotationidx: 3
      classificationclass: color
      pathidx: 0
    GENERATOR: ClassificationGenerator
    GENERATOR_ARGS: {}
  EPOCHS: 10
  FP16: false
  MODEL_SERVING: None
  OPTIMIZER_BUILDER: ClassificationOptimizer
  SKIPEVAL: false
  TEST_FREQUENCY: 5
  TRAINER: ClassificationTrainer
LOGGING:
  STEP_VERBOSE: 100
LOSS:
- KWARGS:
  - {}
  LABEL: color
  LAMBDAS:
  - 1.0
  LOSSES:
  - SoftmaxLogitsLoss
  NAME: out1
LOSS_OPTIMIZER:
- BASE_LR: 0.001
  LR_BIAS_FACTOR: 1.0
  OPTIMIZER: Adam
  OPTIMIZER_KWARGS: {}
  OPTIMIZER_NAME: opt-1
  WEIGHT_BIAS_FACTOR: 0.0005
  WEIGHT_DECAY: 0.0005
LOSS_SCHEDULER:
- LR_KWARGS:
    step_size: 20
  LR_SCHEDULER: StepLR
  SCHEDULER_NAME: opt-1
MODEL:
  BUILDER: classification_model_builde

In [16]:
eml.train()

11:27:39 Starting training
11:27:39 Logging to:	colabel_devel-v1-res18-type-logger.log
11:27:39 Models will be saved to local directory:	colabel_devel-v1-res18-type
11:27:39 Models will be saved with base name:	colabel_devel-v1_epoch[].pth
11:27:39 Optimizers will be saved with base name:	colabel_devel-v1_epoch[]_optimizer.pth
11:27:39 Schedulers will be saved with base name:	colabel_devel-v1_epoch[]_scheduler.pth
11:27:39 Performing initial evaluation...


KeyboardInterrupt: 